# Predicting CitiBike Demand

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load train datasets
df_2024_1 = pd.read_csv('data/2024_08/202408-citibike-tripdata_1.csv', low_memory=False)
df_2024_2 = pd.read_csv('data/2024_08/202408-citibike-tripdata_2.csv', low_memory=False)
df_2024_3 = pd.read_csv('data/2024_08/202408-citibike-tripdata_3.csv', low_memory=False)
df_2024_4 = pd.read_csv('data/2024_08/202408-citibike-tripdata_4.csv', low_memory=False)
df_2024_5 = pd.read_csv('data/2024_08/202408-citibike-tripdata_5.csv', low_memory=False)

In [ ]:
df_2024_1.info()

In [ ]:
df_2024_1.head(12)

In [4]:
# Combine datasets
df_train = pd.concat([df_2024_1, df_2024_2, df_2024_3, df_2024_4, df_2024_5], 
                        ignore_index=True)

In [ ]:
df_train.info()

In [5]:
df_train.to_csv('data/df_train.csv', index=False)